In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
# import catboost as cb
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import lightgbm as lgb
from tqdm import tqdm
from datetime import timedelta

pd.set_option('display.max_columns',None)

In [ ]:
def merge_dfs(df,train,clmns_name):
    df.columns += clmns_name
    train = train.copy().merge(df.drop(['dt' + clmns_name,'day' + clmns_name],axis=1),
            how='left',
            left_on=['month','year'], right_on=['month' + clmns_name,'year' + clmns_name]).set_index(train.index)
    return train


In [ ]:
def get_data_fts(df,date_column):
    df["day"] = df[date_column].dt.day
    df["month"] = df[date_column].dt.month
    df["year"] = df[date_column].dt.year

# PREPROCESSING_TRAIN

In [ ]:
train = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/train.xlsx')
test = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/test.xlsx')
chmf = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/CHMF Акции.csv')
magn = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/MAGN Акции.csv')
nlmk = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/NLMK Акции.csv')
gruz = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Грузоперевозки.xlsx')
materials = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Показатели рынка металла.xlsx')
lme = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Индекс LME.xlsx')
macros = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Макропоказатели.xlsx')
metals = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Показатели рынка металла.xlsx')
fuel = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Топливо.xlsx')
price_raw_material = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Цены на сырье.xlsx')


In [ ]:
def resample_akcii(df,clmn,clmn_name,train=train):
    if df[clmn].dtype == 'object':
        resampled_df = df.loc[df.index >= '2019-01-14',clmn].apply(lambda x: x.replace(",","")).astype('float').resample("7D").mean()
        resampled_df.name = clmn_name
        return resampled_df
    else:
        resampled_df = df.loc[df.index >= '2019-01-14',clmn].resample("7D").mean()
        resampled_df.name = clmn_name
        return resampled_df

def resample_gruz(df,clmn,clmn_name,train=train):
    if df[clmn].dtype == 'object':
        resampled_df = df.loc[df.index >= '2019-01-14',clmn].apply(lambda x: x.replace(",","")).astype('float').resample("7D").mean()
        resampled_df.name = clmn_name
        return resampled_df
    else:
        resampled_df = df.loc[df.index >= '2019-01-14',clmn].resample("7D").mean()
        resampled_df.name = clmn_name
        return resampled_df

In [ ]:
chmf['Date'] = pd.to_datetime(chmf['Date'])
chmf.sort_values(by='Date',inplace=True)

magn['Дата'] = pd.to_datetime(magn['Дата'])
magn.sort_values(by='Дата',inplace=True)

nlmk['Date'] = pd.to_datetime(nlmk['Date'])
nlmk.sort_values(by='Date',inplace=True)

chmf = chmf.set_index('Date')
magn = magn.set_index('Дата')
nlmk = nlmk.set_index('Date')
# gruz = gruz.set_index('dt')

In [ ]:
get_data_fts(train,'dt')
get_data_fts(test,'dt')
get_data_fts(materials,'dt')
get_data_fts(metals,'dt')
get_data_fts(fuel,'dt')

In [ ]:
train = train.set_index('dt')
test = test.set_index('dt')

In [ ]:
gruz.loc[0,'dt'] = pd.to_datetime('2017-09-04') # Изменим одну строку, чтобы без проблем смерджить с трейном

In [ ]:
# Собираем первые три датафрейма с акциями
train = pd.concat([train,
           resample_akcii(chmf,'Open',"chmf_open"),
           resample_akcii(nlmk,'Open',"nlmk_open"),
           resample_akcii(magn,'Откр.',"magn_open"),
           gruz.resample('7D',on='dt').mean(),
           price_raw_material.resample('7D',on='dt').mean(),
           lme.drop(0).resample('7D',on='дата').mean(),
           macros[macros['dt'] >= "2018-01-15"].resample("7D",on='dt').mean()],axis=1).loc[train.index]

train

,Цена на арматуру,day,month,year,chmf_open,nlmk_open,magn_open,Индекс стоимости грузоперевозок,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т","Лом_3А, РФ CPT авто Центральный ФО, руб./т, без НДС","Лом_3А, РФ CPT авто Уральский ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, FOB США Восток, $/т","Лом_3А, FOB РФ Черное море, $/т","Чугун_FOB Бразилия Север, $/т","Лом_3А, РФ CPT авто Южный ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR ЮВА, $/т","Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС","Лом_Б26 легированный (Ni 9,3%), РФ CPT, руб./т, без НДС","ГБЖ_Россия, FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС","Чугун_FOB Бразилия Юг, $/т","ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,78-8,51, сухой вес, DAP Забайкальск-Манжули, $/т","Концентрат коксующегося угля_Россия марка К и смеси марок К, КО FCA руб./т, без НДС","Чугун_FOB Россия Дальневосточные порты, $/т","Кокс_Россия Урал Сибирь +25, FCA руб./т., без НДС","Лом_3А, РФ FCA ж/д Московский регион, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж+ГЖ FCA руб./т, без НДС","Чугун_FOB Россия Черное море, $/т","Лом_3А, РФ FCA ж/д Кемеровская обл, руб./т, без НДС","ЖРС_Россия аглоруда Fe 52-60%, FCA руб./т, без НДС","ЖРС_Россия концентрат Fe 64-68%, FCA руб./т, без НДС","Чугун_Россия, FCA руб./т, без НДС","Концентрат коксующегося угля_HCC Австралия, $/t FOB","ЖРС_Россия окатыши Fe 62-65,5% FCA руб./т, без НДС","ГБЖ_CFR Италия, $/т","Лом_3А, РФ CPT ж/д Уральский ФО, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж FCA руб./т, без НДС","Чугун_CFR Западная Европа, $/т","Лом_3А, РФ FCA ж/д Челябинская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д ХМАО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR Турция, $/т","Чугун_CFR Китай, $/т","Лом_3А, РФ FCA ж/д Самарская обл, руб./т, без НДС","Концентрат коксующегося угля_Средняя шихта Россия FCA руб./т, без НДС","Концентрат коксующегося угля_Россия марка ГЖ FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д Нижегородская обл, руб./т, без НДС","Чугун_CFR NOLA США, $/т","Лом_3А, РФ FCA ж/д Новосибирская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т",цена,Базовый индекс потребительских цен на товары и услуги,Ключевая ставка,Курс доллара
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-05,32900.0,5.0,1.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-12,32500.0,12.0,1.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-19,32000.0,19.0,1.0,2015.0,NaN,NaN,Na

In [ ]:
train = merge_dfs(materials,train,'_materials')
train = merge_dfs(metals,train,'_metals')
train = merge_dfs(fuel,train,'')

train.tail(3)

,Цена на арматуру,day,month,year,chmf_open,nlmk_open,magn_open,Индекс стоимости грузоперевозок,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т","Лом_3А, РФ CPT авто Центральный ФО, руб./т, без НДС","Лом_3А, РФ CPT авто Уральский ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, FOB США Восток, $/т","Лом_3А, FOB РФ Черное море, $/т","Чугун_FOB Бразилия Север, $/т","Лом_3А, РФ CPT авто Южный ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR ЮВА, $/т","Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС","Лом_Б26 легированный (Ni 9,3%), РФ CPT, руб./т, без НДС","ГБЖ_Россия, FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС","Чугун_FOB Бразилия Юг, $/т","ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,78-8,51, сухой вес, DAP Забайкальск-Манжули, $/т","Концентрат коксующегося угля_Россия марка К и смеси марок К, КО FCA руб./т, без НДС","Чугун_FOB Россия Дальневосточные порты, $/т","Кокс_Россия Урал Сибирь +25, FCA руб./т., без НДС","Лом_3А, РФ FCA ж/д Московский регион, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж+ГЖ FCA руб./т, без НДС","Чугун_FOB Россия Черное море, $/т","Лом_3А, РФ FCA ж/д Кемеровская обл, руб./т, без НДС","ЖРС_Россия аглоруда Fe 52-60%, FCA руб./т, без НДС","ЖРС_Россия концентрат Fe 64-68%, FCA руб./т, без НДС","Чугун_Россия, FCA руб./т, без НДС","Концентрат коксующегося угля_HCC Австралия, $/t FOB","ЖРС_Россия окатыши Fe 62-65,5% FCA руб./т, без НДС","ГБЖ_CFR Италия, $/т","Лом_3А, РФ CPT ж/д Уральский ФО, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж FCA руб./т, без НДС","Чугун_CFR Западная Европа, $/т","Лом_3А, РФ FCA ж/д Челябинская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д ХМАО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR Турция, $/т","Чугун_CFR Китай, $/т","Лом_3А, РФ FCA ж/д Самарская обл, руб./т, без НДС","Концентрат коксующегося угля_Средняя шихта Россия FCA руб./т, без НДС","Концентрат коксующегося угля_Россия марка ГЖ FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д Нижегородская обл, руб./т, без НДС","Чугун_CFR NOLA США, $/т","Лом_3А, РФ FCA ж/д Новосибирская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т",цена,Базовый индекс потребительских цен на товары и услуги,Ключевая ставка,Курс доллара,Производство арматуры Сортовой прокат_Арматура_materials,Производство арматуры Сортовой прокат_Катанка_materials,Производство арматуры Сортовой прокат_Прутки прочие_materials,Производство сырья ЖРС_Агломерат_materials,Производство сырья ЖРС_Аглоруда_materials,Производство сырья ЖРС_Концентрат_materials,Производство сырья ЖРС_Концентрат для дообогащения_materials,Производство сырья ЖРС_Окатыши_materials,Производство сырья Кокс_Кокс валовый_materials,Производство сырья Концентрат коксующегося угля_Жирные_materials,Производство сырья Концентрат коксующегося угля_Коксовые_materials,Производство сырья Концентрат коксующегося угля_Отощающие_materials,Производство сырья Металлизованная продукция_ГБЖ_materials,Про

In [ ]:
# train = train[train.index > '2019-01-01']
train = train.drop(train.columns[train.isna().sum() > 39], axis=1).dropna()

In [ ]:
# train.to_parquet('/content/drive/MyDrive/SAMOLET_CASE/data/train_all_fts.pqt',index=True)

# PREPROCESSING_TEST

In [ ]:
train = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/train.xlsx')
test = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/test.xlsx')
chmf = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/CHMF Акции.csv')
magn = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/MAGN Акции.csv')
nlmk = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/NLMK Акции.csv')
gruz = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Грузоперевозки.xlsx')
materials = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Показатели рынка металла.xlsx')
lme = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Индекс LME.xlsx')
macros = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Макропоказатели.xlsx')
metals = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Показатели рынка металла.xlsx')
fuel = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Топливо.xlsx')
price_raw_material = pd.read_excel('/content/drive/MyDrive/SAMOLET_CASE/data/Цены на сырье.xlsx')


In [ ]:
chmf['Date'] = pd.to_datetime(chmf['Date'])
chmf.sort_values(by='Date',inplace=True)

magn['Дата'] = pd.to_datetime(magn['Дата'])
magn.sort_values(by='Дата',inplace=True)

nlmk['Date'] = pd.to_datetime(nlmk['Date'])
nlmk.sort_values(by='Date',inplace=True)

chmf = chmf.set_index('Date')
magn = magn.set_index('Дата')
nlmk = nlmk.set_index('Date')
# gruz = gruz.set_index('dt')

In [ ]:
get_data_fts(train,'dt')
get_data_fts(test,'dt')
get_data_fts(materials,'dt')
get_data_fts(metals,'dt')
get_data_fts(fuel,'dt')

In [ ]:
train = train.set_index('dt')
test = test.set_index('dt')

In [ ]:
gruz.loc[0,'dt'] = pd.to_datetime('2017-09-04') # Изменим одну строку, чтобы без проблем смерджить с трейном

In [ ]:
# Собираем первые три датафрейма с акциями
test = pd.concat([test,
           resample_akcii(chmf,'Open',"chmf_open"),
           resample_akcii(nlmk,'Open',"nlmk_open"),
           resample_akcii(magn,'Откр.',"magn_open"),
           gruz.resample('7D',on='dt').mean(),
           price_raw_material.resample('7D',on='dt').mean(),
           lme.drop(0).resample('7D',on='дата').mean(),
           macros[macros['dt'] >= "2018-01-15"].resample("7D",on='dt').mean()],axis=1).loc[test.index]

test

,Цена на арматуру,day,month,year,chmf_open,nlmk_open,magn_open,Индекс стоимости грузоперевозок,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т","Лом_3А, РФ CPT авто Центральный ФО, руб./т, без НДС","Лом_3А, РФ CPT авто Уральский ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, FOB США Восток, $/т","Лом_3А, FOB РФ Черное море, $/т","Чугун_FOB Бразилия Север, $/т","Лом_3А, РФ CPT авто Южный ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR ЮВА, $/т","Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС","Лом_Б26 легированный (Ni 9,3%), РФ CPT, руб./т, без НДС","ГБЖ_Россия, FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС","Чугун_FOB Бразилия Юг, $/т","ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,78-8,51, сухой вес, DAP Забайкальск-Манжули, $/т","Концентрат коксующегося угля_Россия марка К и смеси марок К, КО FCA руб./т, без НДС","Чугун_FOB Россия Дальневосточные порты, $/т","Кокс_Россия Урал Сибирь +25, FCA руб./т., без НДС","Лом_3А, РФ FCA ж/д Московский регион, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж+ГЖ FCA руб./т, без НДС","Чугун_FOB Россия Черное море, $/т","Лом_3А, РФ FCA ж/д Кемеровская обл, руб./т, без НДС","ЖРС_Россия аглоруда Fe 52-60%, FCA руб./т, без НДС","ЖРС_Россия концентрат Fe 64-68%, FCA руб./т, без НДС","Чугун_Россия, FCA руб./т, без НДС","Концентрат коксующегося угля_HCC Австралия, $/t FOB","ЖРС_Россия окатыши Fe 62-65,5% FCA руб./т, без НДС","ГБЖ_CFR Италия, $/т","Лом_3А, РФ CPT ж/д Уральский ФО, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж FCA руб./т, без НДС","Чугун_CFR Западная Европа, $/т","Лом_3А, РФ FCA ж/д Челябинская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д ХМАО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR Турция, $/т","Чугун_CFR Китай, $/т","Лом_3А, РФ FCA ж/д Самарская обл, руб./т, без НДС","Концентрат коксующегося угля_Средняя шихта Россия FCA руб./т, без НДС","Концентрат коксующегося угля_Россия марка ГЖ FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д Нижегородская обл, руб./т, без НДС","Чугун_CFR NOLA США, $/т","Лом_3А, РФ FCA ж/д Новосибирская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т",цена,Базовый индекс потребительских цен на товары и услуги,Ключевая ставка,Курс доллара
dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-05,46000.0,5.0,9.0,2022.0,785.36,121.300,2913.250000,1274.0,7750.0,359.0,24677.0,20920.0,23357.0,98.0,435.0,6250.0,108.5,22150.0,21250.0,368.0,351.0,440.0,22350.0,440.0,5750.0,NaN,NaN,22383.0,NaN,65.5,10500.0,415.0,14500.0,21498.0,7950.0,370.0,NaN,2025.0,3800.0,23350.0,270.0,5050.0,397.5,24665.0,10500.0,NaN,NaN,NaN,396.0,425.0,NaN,7840.0,5400.0,NaN,465.0,NaN,NaN,400.0,4507.0,3470.0,1950.0,39.0,52.0,71.0,98.0,163.0,670.000,NaN,8.0,60.841940
2022-09-12,46000.0,12.0,9.0,2022.0,776.64,118.612,2998.800000,1266.0,7750.0,339.0,24588.0,21739.0,22647.0,99.0,425.0,6250.0,108.5,22000.0,21000.0,342.0,328.0,440.0,21850.0,415.0,5750.0,NaN,N

In [ ]:
test = merge_dfs(materials,test,'_materials')
test = merge_dfs(metals,test,'_metals')
test = merge_dfs(fuel,test,'')

test.tail(3)

,Цена на арматуру,day,month,year,chmf_open,nlmk_open,magn_open,Индекс стоимости грузоперевозок,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т","Лом_3А, РФ CPT авто Центральный ФО, руб./т, без НДС","Лом_3А, РФ CPT авто Уральский ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, FOB США Восток, $/т","Лом_3А, FOB РФ Черное море, $/т","Чугун_FOB Бразилия Север, $/т","Лом_3А, РФ CPT авто Южный ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR ЮВА, $/т","Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС","Лом_Б26 легированный (Ni 9,3%), РФ CPT, руб./т, без НДС","ГБЖ_Россия, FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС","Чугун_FOB Бразилия Юг, $/т","ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,78-8,51, сухой вес, DAP Забайкальск-Манжули, $/т","Концентрат коксующегося угля_Россия марка К и смеси марок К, КО FCA руб./т, без НДС","Чугун_FOB Россия Дальневосточные порты, $/т","Кокс_Россия Урал Сибирь +25, FCA руб./т., без НДС","Лом_3А, РФ FCA ж/д Московский регион, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж+ГЖ FCA руб./т, без НДС","Чугун_FOB Россия Черное море, $/т","Лом_3А, РФ FCA ж/д Кемеровская обл, руб./т, без НДС","ЖРС_Россия аглоруда Fe 52-60%, FCA руб./т, без НДС","ЖРС_Россия концентрат Fe 64-68%, FCA руб./т, без НДС","Чугун_Россия, FCA руб./т, без НДС","Концентрат коксующегося угля_HCC Австралия, $/t FOB","ЖРС_Россия окатыши Fe 62-65,5% FCA руб./т, без НДС","ГБЖ_CFR Италия, $/т","Лом_3А, РФ CPT ж/д Уральский ФО, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж FCA руб./т, без НДС","Чугун_CFR Западная Европа, $/т","Лом_3А, РФ FCA ж/д Челябинская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д ХМАО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR Турция, $/т","Чугун_CFR Китай, $/т","Лом_3А, РФ FCA ж/д Самарская обл, руб./т, без НДС","Концентрат коксующегося угля_Средняя шихта Россия FCA руб./т, без НДС","Концентрат коксующегося угля_Россия марка ГЖ FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д Нижегородская обл, руб./т, без НДС","Чугун_CFR NOLA США, $/т","Лом_3А, РФ FCA ж/д Новосибирская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т",цена,Базовый индекс потребительских цен на товары и услуги,Ключевая ставка,Курс доллара,Производство арматуры Сортовой прокат_Арматура_materials,Производство арматуры Сортовой прокат_Катанка_materials,Производство арматуры Сортовой прокат_Прутки прочие_materials,Производство сырья ЖРС_Агломерат_materials,Производство сырья ЖРС_Аглоруда_materials,Производство сырья ЖРС_Концентрат_materials,Производство сырья ЖРС_Концентрат для дообогащения_materials,Производство сырья ЖРС_Окатыши_materials,Производство сырья Кокс_Кокс валовый_materials,Производство сырья Концентрат коксующегося угля_Жирные_materials,Производство сырья Концентрат коксующегося угля_Коксовые_materials,Производство сырья Концентрат коксующегося угля_Отощающие_materials,Производство сырья Металлизованная продукция_ГБЖ_materials,Про

In [ ]:
# test.to_parquet('/content/drive/MyDrive/SAMOLET_CASE/data/test_all_fts.pqt',index=True)